# Module 11: Packaging, Reuse & ML Deployment Lite

**Learning Objectives:**
- Scaffold and structure an analytics/ML Python package
- Write reusable utility modules for wrangling and validation
- Save and load ML models using `joblib`/`pickle`
- Write documentation and reusable templates
- Build simple CLI interfaces for automated pipeline runs

> **Why does this matter?** Notebooks are great for exploration, but production data systems are built from reusable modules, deployed as scripts, and served via APIs or CLIs. This module bridges the gap between a notebook prototype and a working system.

In [ ]:
import os
import sys
import json
import time
import pickle
import joblib
import argparse
import textwrap
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, accuracy_score

os.makedirs('models', exist_ok=True)
os.makedirs('src', exist_ok=True)
print("Environment ready!")

---
## 1. Project Scaffolding for Analytics + ML

A well-structured Python package makes your code importable, testable, and shareable.

In [ ]:
# Write a clean src package structure
src = Path('src')

# __init__.py
(src / '__init__.py').write_text(textwrap.dedent("""
# src/__init__.py
# Entry point for the analytics package.
from .clean import clean_dataframe
from .validate import DataValidator
from .model import train_model, predict
""").strip())

print("src/__init__.py written.")

---
## 2. Utility Module: Wrangling (`src/clean.py`)

In [ ]:
clean_module = textwrap.dedent("""
# src/clean.py
"""Reusable data cleaning utilities."""

import numpy as np
import pandas as pd
from typing import Optional, List, Dict


def strip_whitespace(df: pd.DataFrame, columns: Optional[List[str]] = None) -> pd.DataFrame:
    """Strip leading/trailing whitespace from string columns."""
    cols = columns or df.select_dtypes(include='object').columns.tolist()
    for col in cols:
        df[col] = df[col].str.strip()
    return df


def drop_duplicates_logged(df: pd.DataFrame, subset=None, logger=None) -> pd.DataFrame:
    """Drop duplicates and log the count removed."""
    before = len(df)
    df = df.drop_duplicates(subset=subset)
    removed = before - len(df)
    msg = f"Removed {removed} duplicate rows."
    if logger:
        logger.info(msg)
    else:
        print(msg)
    return df


def impute_columns(df: pd.DataFrame, strategy: Dict[str, str]) -> pd.DataFrame:
    """
    Impute missing values per column.
    strategy: dict of {column: 'mean' | 'median' | 'mode' | <literal>}
    """
    for col, method in strategy.items():
        if col not in df.columns:
            continue
        if method == 'mean':
            df[col] = df[col].fillna(df[col].mean())
        elif method == 'median':
            df[col] = df[col].fillna(df[col].median())
        elif method == 'mode':
            df[col] = df[col].fillna(df[col].mode()[0])
        else:
            df[col] = df[col].fillna(method)
    return df


def encode_categoricals(df: pd.DataFrame, columns: List[str], drop_first=True) -> pd.DataFrame:
    """One-hot encode a list of categorical columns."""
    return pd.get_dummies(df, columns=columns, drop_first=drop_first)


def clean_dataframe(df: pd.DataFrame, config: dict) -> pd.DataFrame:
    """
    Master cleaning function driven by a config dict.
    config keys: strip_cols, dedup_cols, impute, encode_cols
    """
    df = df.copy()
    if config.get('strip_cols'):
        df = strip_whitespace(df, config['strip_cols'])
    if config.get('dedup_cols') is not None:
        df = drop_duplicates_logged(df, subset=config['dedup_cols'] or None)
    if config.get('impute'):
        df = impute_columns(df, config['impute'])
    if config.get('encode_cols'):
        df = encode_categoricals(df, config['encode_cols'])
    return df
""")

(src / 'clean.py').write_text(clean_module)
print("src/clean.py written.")

---
## 3. Utility Module: Validation (`src/validate.py`)

In [ ]:
validate_module = textwrap.dedent("""
# src/validate.py
"""Reusable data validation utilities."""

import numpy as np
import pandas as pd
from typing import List, Optional


class DataValidator:
    """Chainable data validation class."""

    def __init__(self, df: pd.DataFrame):
        self.df = df
        self.errors: dict = {}
        self.warnings: dict = {}

    def required_columns(self, columns: List[str]) -> 'DataValidator':
        """Check that required columns exist."""
        missing = [c for c in columns if c not in self.df.columns]
        if missing:
            self.errors['missing_columns'] = f"Columns not found: {missing}"
        return self

    def no_nulls(self, columns: List[str]) -> 'DataValidator':
        """Check that columns have no null values."""
        for col in columns:
            count = self.df[col].isnull().sum()
            if count > 0:
                self.errors[f'{col}_nulls'] = f"{count} null values found."
        return self

    def value_range(self, col: str, min_val=None, max_val=None) -> 'DataValidator':
        """Check numeric values fall within [min_val, max_val]."""
        if min_val is not None and (self.df[col] < min_val).any():
            self.errors[f'{col}_below_min'] = f"Values below {min_val} found."
        if max_val is not None and (self.df[col] > max_val).any():
            self.warnings[f'{col}_above_max'] = f"Values above {max_val} found."
        return self

    def allowed_values(self, col: str, allowed: list) -> 'DataValidator':
        """Check all values are in the allowed set."""
        invalid = self.df[~self.df[col].isin(allowed)][col].unique()
        if len(invalid):
            self.errors[f'{col}_invalid_values'] = f"Unexpected values: {invalid.tolist()}"
        return self

    def row_count(self, min_rows: int) -> 'DataValidator':
        """Check that DataFrame has at least min_rows rows."""
        if len(self.df) < min_rows:
            self.errors['row_count'] = f"Expected >= {min_rows} rows, got {len(self.df)}."
        return self

    def generate_report(self) -> str:
        """Return a formatted validation summary."""
        lines = ['=== Data Validation Report ===']
        if not self.errors and not self.warnings:
            lines.append('✅ All checks passed!')
        for k, v in self.errors.items():
            lines.append(f'❌ ERROR   [{k}]: {v}')
        for k, v in self.warnings.items():
            lines.append(f'⚠️  WARNING [{k}]: {v}')
        return '\n'.join(lines)

    def is_valid(self) -> bool:
        return len(self.errors) == 0
""")

(src / 'validate.py').write_text(validate_module)
print("src/validate.py written.")

---
## 4. Utility Module: Model (`src/model.py`)

In [ ]:
model_module = textwrap.dedent("""
# src/model.py
"""ML training, saving, and prediction utilities."""

import joblib
import numpy as np
import pandas as pd
from typing import List
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error


def build_regression_pipeline(estimator=None):
    """Build a sklearn Pipeline: scaler + estimator."""
    estimator = estimator or LinearRegression()
    return Pipeline([
        ('scaler', StandardScaler()),
        ('model', estimator)
    ])


def train_model(df: pd.DataFrame, feature_cols: List[str], target_col: str,
                model_path: str = 'models/model.pkl', test_size=0.2, random_state=42):
    """Train, evaluate, and save a regression model."""
    X = df[feature_cols]
    y = df[target_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    pipeline = build_regression_pipeline()
    pipeline.fit(X_train, y_train)

    y_pred = pipeline.predict(X_test)
    metrics = {
        'r2': r2_score(y_test, y_pred),
        'rmse': np.sqrt(mean_squared_error(y_test, y_pred)),
        'n_train': len(X_train),
        'n_test': len(X_test)
    }

    Path(model_path).parent.mkdir(parents=True, exist_ok=True)
    joblib.dump(pipeline, model_path)

    return pipeline, metrics


def predict(input_df: pd.DataFrame, model_path: str) -> np.ndarray:
    """Load a saved model and make predictions."""
    pipeline = joblib.load(model_path)
    return pipeline.predict(input_df)
""")

(src / 'model.py').write_text(model_module)
print("src/model.py written.")

---
## 5. Exporting ML Models: joblib vs pickle

| Method | Best For | Pros | Cons |
|---|---|---|---|
| `joblib.dump` | sklearn models (especially with numpy arrays) | Faster for large arrays, compression support | joblib-only |
| `pickle` | Any Python object | Universal | Slower for large arrays |
| `model.save()` | TensorFlow/Keras | Native format | Framework-specific |

In [ ]:
# Train a model to demonstrate saving/loading
np.random.seed(42)
n = 1000
demo_df = pd.DataFrame({
    'years_exp': np.random.randint(0, 30, n),
    'performance': np.random.randint(50, 100, n),
    'education': np.random.choice([0, 1, 2], n),
    'salary': 30000 + np.random.randint(0, 30, n) * 2500 + np.random.normal(0, 4000, n)
})

feature_cols = ['years_exp', 'performance', 'education']
X = demo_df[feature_cols]
y = demo_df['salary']

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42)

# Build and train a sklearn Pipeline (scaler + model combined)
pipe = Pipeline([('scaler', StandardScaler()), ('model', LinearRegression())])
pipe.fit(X_tr, y_tr)

r2 = r2_score(y_te, pipe.predict(X_te))
print(f"Pipeline trained. R² on test: {r2:.4f}")

In [ ]:
# Save with joblib (recommended for sklearn)
joblib_path = 'models/salary_pipeline.pkl'
joblib.dump(pipe, joblib_path, compress=3)
print(f"Saved with joblib: {joblib_path} ({os.path.getsize(joblib_path)} bytes)")

# Save with pickle
pickle_path = 'models/salary_pipeline_pickle.pkl'
with open(pickle_path, 'wb') as f:
    pickle.dump(pipe, f)
print(f"Saved with pickle: {pickle_path} ({os.path.getsize(pickle_path)} bytes)")

In [ ]:
# Load and use the saved pipeline
loaded_pipe = joblib.load(joblib_path)

# Predict on new data
new_candidate = pd.DataFrame({'years_exp': [10], 'performance': [88], 'education': [2]})
predicted_salary = loaded_pipe.predict(new_candidate)[0]
print(f"Predicted salary for new candidate: ${predicted_salary:,.2f}")

---
## 6. Model Versioning & Metadata

Always save metadata alongside your model so you know exactly what was used.

In [ ]:
def save_model_with_metadata(pipeline, metrics, feature_cols, model_dir='models', name='salary_model'):
    """Save a model pipeline alongside a JSON metadata file."""
    os.makedirs(model_dir, exist_ok=True)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    model_path = os.path.join(model_dir, f"{name}_{timestamp}.pkl")
    meta_path  = os.path.join(model_dir, f"{name}_{timestamp}_meta.json")
    latest_path = os.path.join(model_dir, f"{name}_latest.pkl")

    joblib.dump(pipeline, model_path, compress=3)
    joblib.dump(pipeline, latest_path, compress=3)  # Always overwrite 'latest'

    metadata = {
        'model_path': model_path,
        'trained_at': timestamp,
        'features': feature_cols,
        'metrics': metrics,
        'pipeline_steps': [s[0] for s in pipeline.steps]
    }
    with open(meta_path, 'w') as f:
        json.dump(metadata, f, indent=2)

    print(f"✅ Model saved: {model_path}")
    print(f"📋 Metadata:   {meta_path}")
    return metadata

metrics = {'r2': r2, 'rmse': 5000.0, 'n_train': 800, 'n_test': 200}
meta = save_model_with_metadata(pipe, metrics, feature_cols)
print(json.dumps(meta, indent=2))

---
## 7. Building a Command-Line Interface (CLI)

A CLI lets your pipeline be triggered from a terminal, cron job, or n8n.

In [ ]:
# Write a complete CLI pipeline script
cli_script = textwrap.dedent("""
#!/usr/bin/env python
# scripts/run_pipeline.py
"""
Full ML pipeline CLI.
Usage:
  python run_pipeline.py --input data/raw/employees.csv --output models/ --target salary
  python run_pipeline.py --input data/raw/employees.csv --target salary --mode predict --model models/salary_model_latest.pkl
"""

import argparse
import json
import sys
import logging
import pandas as pd
import numpy as np
import joblib
from pathlib import Path
from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score


logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(message)s',
    handlers=[logging.StreamHandler(), logging.FileHandler('logs/pipeline.log')]
)
log = logging.getLogger('pipeline')


def parse_args():
    parser = argparse.ArgumentParser(description='ML Pipeline Runner')
    parser.add_argument('--input',   required=True,  help='Path to input CSV file')
    parser.add_argument('--target',  required=True,  help='Target column name')
    parser.add_argument('--output',  default='models', help='Output directory for models')
    parser.add_argument('--mode',    choices=['train', 'predict'], default='train')
    parser.add_argument('--model',   default=None,   help='Path to saved model (for predict mode)')
    parser.add_argument('--test-size', type=float,   default=0.2)
    return parser.parse_args()


def train_pipeline(df, target_col, test_size, output_dir):
    feature_cols = [c for c in df.select_dtypes(include=[np.number]).columns if c != target_col]
    X, y = df[feature_cols], df[target_col]
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=test_size, random_state=42)

    log.info(f"Training on {len(X_tr)} samples, testing on {len(X_te)} samples.")
    pipe = Pipeline([('scaler', StandardScaler()), ('model', LinearRegression())])
    pipe.fit(X_tr, y_tr)

    r2 = r2_score(y_te, pipe.predict(X_te))
    log.info(f"R² score: {r2:.4f}")

    Path(output_dir).mkdir(parents=True, exist_ok=True)
    ts = datetime.now().strftime('%Y%m%d_%H%M%S')
    model_path = f"{output_dir}/model_{ts}.pkl"
    joblib.dump(pipe, model_path)
    joblib.dump(pipe, f"{output_dir}/model_latest.pkl")
    log.info(f"Model saved to {model_path}")

    print(json.dumps({'status': 'success', 'r2': r2, 'model_path': model_path}))


def predict_pipeline(df, target_col, model_path):
    feature_cols = [c for c in df.select_dtypes(include=[np.number]).columns if c != target_col]
    pipe = joblib.load(model_path)
    predictions = pipe.predict(df[feature_cols])
    df['prediction'] = predictions
    output = df[['prediction']].to_dict(orient='records')
    print(json.dumps({'status': 'success', 'predictions': output[:5], 'total': len(output)}))


def main():
    args = parse_args()
    log.info(f"Loading data from {args.input}")
    df = pd.read_csv(args.input)
    log.info(f"Loaded {len(df)} rows, {len(df.columns)} columns.")

    if args.mode == 'train':
        train_pipeline(df, args.target, args.test_size, args.output)
    elif args.mode == 'predict':
        if not args.model:
            log.error("--model is required for predict mode.")
            sys.exit(1)
        predict_pipeline(df, args.target, args.model)


if __name__ == '__main__':
    main()
""")

os.makedirs('scripts', exist_ok=True)
with open('scripts/run_pipeline.py', 'w') as f:
    f.write(cli_script)

print("✅ scripts/run_pipeline.py written!")
print("\nUsage examples:")
print("  python scripts/run_pipeline.py --input data/raw/employees.csv --target salary")
print("  python scripts/run_pipeline.py --input data/raw/employees.csv --target salary --mode predict --model models/model_latest.pkl")

---
## 8. Documentation: Auto-generating Docstrings & READMEs

In [ ]:
# README template generator
def generate_readme(project_name, description, features, usage_example, output_path='README.md'):
    features_md = '\n'.join(f'- {f}' for f in features)
    readme = f"""# {project_name}

{description}

## Features

{features_md}

## Installation

```bash
python -m venv venv
source venv/bin/activate  # Windows: venv\\Scripts\\activate
pip install -r requirements.txt
```

## Usage

```bash
{usage_example}
```

## Project Structure

```
src/         Reusable modules
scripts/     CLI pipeline runners
data/        Raw and processed datasets
models/      Saved ML model artifacts
logs/        Pipeline execution logs
```

## License

MIT
"""
    with open(output_path, 'w') as f:
        f.write(readme)
    print(f"README written to {output_path}")
    return readme

readme = generate_readme(
    'Salary Prediction Pipeline',
    'An end-to-end ML pipeline for predicting employee salaries from HR data.',
    [
        'Chunked CSV ingestion for large files',
        'Configurable data cleaning and validation',
        'Sklearn Pipeline with StandardScaler + LinearRegression',
        'CLI interface for training and prediction',
        'Model versioning with metadata JSON'
    ],
    'python scripts/run_pipeline.py --input data/raw/employees.csv --target salary'
)
print(readme[:800])

---
## 9. Reusable Pipeline Template

The master pattern for any production ML pipeline:

In [ ]:
# ============================================================
# REUSABLE ML PIPELINE TEMPLATE
# ============================================================

PIPELINE_CONFIG = {
    'data_path': 'data/raw/employees.csv',
    'target_col': 'salary',
    'feature_cols': ['years_exp', 'performance', 'education'],
    'clean': {
        'impute': {'years_exp': 'median', 'performance': 'mean'},
        'encode_cols': []  # categorical cols to one-hot encode
    },
    'model_dir': 'models',
    'test_size': 0.2
}

def run_ml_pipeline(config: dict):
    """Execute the full ML pipeline from config."""
    print("\n" + "="*50)
    print(f"Pipeline started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("="*50)

    # 1. Load data
    print("[1/5] Loading data...")
    demo_df = pd.DataFrame({
        'years_exp': np.random.randint(0, 30, 1000),
        'performance': np.random.randint(50, 100, 1000),
        'education': np.random.choice([0, 1, 2], 1000),
        'salary': 30000 + np.random.randint(0, 30, 1000) * 2500
    })

    # 2. Validate
    print("[2/5] Validating...")
    from src.validate import DataValidator
    v = (DataValidator(demo_df)
         .required_columns(config['feature_cols'] + [config['target_col']])
         .no_nulls(config['feature_cols'])
         .row_count(50))
    print(v.generate_report())
    if not v.is_valid():
        raise ValueError("Validation failed. Aborting pipeline.")

    # 3. Feature engineering (basic)
    print("[3/5] Feature engineering...")
    X = demo_df[config['feature_cols']]
    y = demo_df[config['target_col']]

    # 4. Train
    print("[4/5] Training model...")
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=config['test_size'], random_state=42)
    pipe = Pipeline([('scaler', StandardScaler()), ('model', LinearRegression())])
    pipe.fit(X_tr, y_tr)
    r2 = r2_score(y_te, pipe.predict(X_te))
    print(f"   R² = {r2:.4f}")

    # 5. Save
    print("[5/5] Saving model...")
    os.makedirs(config['model_dir'], exist_ok=True)
    model_path = os.path.join(config['model_dir'], 'model_latest.pkl')
    joblib.dump(pipe, model_path)
    print(f"   Saved to: {model_path}")

    print("\n✅ Pipeline complete!")
    return {'r2': r2, 'model_path': model_path}

run_ml_pipeline(PIPELINE_CONFIG)

---
## 10. Exercises

1. Add a `type_check` method to `DataValidator` that verifies column dtypes match expected types.
2. Extend `save_model_with_metadata` to also log the Python version, sklearn version, and dataset row count.
3. Modify `run_pipeline.py` to support `--config path/to/config.json` so all settings can be driven from a file.
4. Write a `src/report.py` module with a `generate_model_card(metadata)` function that prints a formatted summary of the model.
5. Package the `src/` module by writing a `setup.py` or `pyproject.toml`. Install it with `pip install -e .` and import it in a fresh Python session.